In [28]:
import pandas as pd
import numpy as np
import os
import pinyin

In [56]:
# reading in the data 
root = os.path.dirname(os.getcwd())
data_dir = os.path.join(root, 'data')
time_series_7_day_dir = os.path.join(data_dir, 'time_series', '7_day_moving_average')
time_series_path = os.path.join(time_series_7_day_dir, 'df_m7.csv')
snapshot_path = os.path.join(data_dir, 'snapshot_data.csv')
snapshots = pd.read_csv(snapshot_path)

In [38]:
# generate summary statistics
column_name_map = {
                   'POP':'Population (in thousands)',
                   'Area':'City area (in km^2)',
                   'POPDENS':'Population Density (people per km^2)',
                   'GDP':'GDP (Billions USD)',
                   'PRIM':'Primary sector (Billions USD)',
                   'SEC':'Secondary sector (Billions USD)',
                   'TERT':'Tertiary sector (Billions USD)',
                   'Prim%':'Primary sector % of GDP',
                   'Sec%':'Secondary sector % of GDP',
                   'Tert%':'Tertiary sector % of GDP',
                   'GDPpc':'GDP per capita (Billions USD per km^2)',
                   '>60yr%':'Elderly population %',
                   'BED':'Hospital Beds (per thousand people)',
                   'DOC':'Registered doctors (per thousand)',
                   'NRS':'Registered nurses (per thousand)',
                   'TVLR':'Wuhan travellers (thousands)',
                   'TVLR‰':'Wuhan travellers (per thousand pop.)',
                   'ACTV':'Average degree of activeness (0-8)'
                  }
'', 'std', 'amin', 'percentile', 'median', 'percentile', 'amax'
snapshots['POP'] *= 10
snapshots['POPDENS'] *= 10000
snapshots['Prim%'] *= 100
snapshots['Sec%'] *= 100
snapshots['Tert%'] *= 100
snapshots['>60yr%'] *= 100
snapshots['TVLR'] *= 10
snapshots['TVLR‰'] *= 1000
snapshots = snapshots.drop(
    columns=[x for x in snapshots.columns if x not in column_name_map]).rename(
    column_name_map, axis=1)
first_quartile = partial(np.percentile, q=25, axis=0)
third_quartile = partial(np.percentile, q=75, axis=0)
summary_stats_snapshot  = snapshots.apply([np.mean, np.std, np.min, first_quartile,
                 np.median, third_quartile, np.max],
                 axis=0, result_type='broadcast').T.round(2)
summary_stats_snapshot.columns = ['Mean',
                         'SD',
                         'Min',
                         '25th %ile',
                         'Median',
                         '75th %ile',
                         'Max']
summary_stats_snapshot['IQR'] = summary_stats_snapshot['75th %ile'] - summary_stats_snapshot['25th %ile']
summary_stats_snapshot

,Mean,SD,Min,25th %ile,Median,75th %ile,Max,IQR
Population (in thousands),5624.67,4029.79,720.96,3176.92,4666.55,7181.67,31243.20,4004.75
GDP (Billions USD),4561.69,5633.80,354.72,1599.24,2759.78,4979.38,38155.32,3380.14
Primary sector (Billions USD),244.45,173.23,11.80,133.27,216.35,321.06,1511.42,187.79
Secondary sector (Billions USD),1782.75,1861.46,130.54,704.88,1120.20,2072.60,10495.84,1367.72
Tertiary sector (Billions USD),2534.45,3942.48,197.22,796.67,1319.62,2465.34,29542.50,1668.67
Elderly population %,19.51,4.51,4.92,17.13,19.69,22.48,32.20,5.35
Hospital Beds (per thousand people),6.22,1.22,3.82,5.43,6.10,6.90,9.67,1.47
Registered doctors (per thousand),2.81,0.76,1.32,2.29,2.73,3.14,5.76,0.85
Registered nurses (per thousand),3.19,1.01,1.27,2.51,3.03,3.60,6.72,1.09
City area (in km^2),11733.64,9080.77,1459.00,6339.50,10238.00,14288.50,82402.00,7949.00


In [76]:
column_name_map_2 = {
                        'Date':'Date',
                        'PM2.5':'PM2.5 (μg/m3)',
                        'PM10':'PM10 (μg/m3)',
                        'SO2':'SO2 (μg/m3)',
                        'CO':'CO (mg/m3)' ,
                        'NO2':'NO2 (μg/m3)',
                        'O3':'O3 (μg/m3)',
                        'HUM':'Relative humidity (%)',
                        'PRES':'Atmospheric pressure (hpa)',
                        'WSPD':'Wind speed (m/s)',
                        'TEMP':'Average air temperature',
                        'ACTV':'Degree of activeness',
                        'Case':'New confirmed cases',
                        'MORB%':'Morbidity rate'
                    }

def eng_translator(phrase_list):
    translation_map = {}
    for phrase in phrase_list:
        pinyin_phrase = pinyin.get(phrase, format="strip", delimiter=" ")
        pinyin_phrase = ''.join(pinyin_phrase.split(' '))
        pinyin_phrase = pinyin_phrase[0].upper() + pinyin_phrase[1:]
        translation_map[phrase] = pinyin_phrase
    return translation_map

def translate_column_to_eng(df, col):
    unique_phrases = np.unique(df[col]).tolist()
    translation_map = eng_translator(unique_phrases)
    return df[col].map(translation_map)

time_series = pd.read_csv(time_series_path)
time_series.columns.values[0] = 'Date'
time_series.columns.values[1] = 'City'
time_series['City'] = translate_column_to_eng(time_series, 'City')
aggregated_time_series = time_series.groupby('City').mean().reset_index()
aggregated_time_series = aggregated_time_series.drop(
    columns=[x for x in time_series.columns if x not in column_name_map_2]).rename(
    column_name_map_2, axis=1)
summary_stats_time_series = aggregated_time_series.apply([np.mean, np.std, np.min, first_quartile,
                 np.median, third_quartile, np.max],
                 axis=0, result_type='broadcast').T.round(2)
summary_stats_time_series.columns = ['Mean',
                         'SD',
                         'Min',
                         '25th %ile',
                         'Median',
                         '75th %ile',
                         'Max']
summary_stats_time_series

,Mean,SD,Min,25th %ile,Median,75th %ile,Max
PM2.5 (μg/m3),48.17,16.68,9.57,35.76,45.60,62.26,89.15
PM10 (μg/m3),71.34,24.91,24.00,51.47,67.44,92.70,121.52
SO2 (μg/m3),10.43,6.19,3.13,6.23,8.11,12.77,36.26
CO (mg/m3),0.83,0.21,0.49,0.68,0.79,0.92,1.63
NO2 (μg/m3),25.20,6.86,7.39,20.88,24.79,30.19,47.00
O3 (μg/m3),82.60,8.84,56.32,77.61,83.82,88.73,103.79
Relative humidity (%),71.57,12.06,24.36,63.51,74.95,80.52,89.74
Atmospheric pressure (hpa),991.44,50.43,649.73,982.15,1012.55,1018.59,1023.16
Wind speed (m/s),2.23,1.01,0.36,1.57,2.04,2.52,5.46
Average air temperature,8.76,4.74,-7.28,6.80,8.89,11.26,22.58


### Notes
- The original data doesn't measure in same units as summary table 
    - Needed to convert to in thousands and proportions to percentages 

- The GDP measurements are off (probably due to different data being used)
    - GDP
    - Primary sector
    - Secondary sector 
    - Tertiary sector 
    - Primary sector %
    - Secondary sector %
    - Tertiary sector %
    - GDP per capita
    
- Time series measurements are difficult to replicate
    - Need to figure out if I aggregate over all dates, then over all cities

- Need to append ```\usepackage{booktabs}``` before using LaTeX representation of table

In [ ]:
print(summary_stats_snapshot.to_latex(index = True, multirow = True, bold_rows=True))

In [ ]:
print(summary_stats_time_series.to_latex(index = True, multirow = True, bold_rows=True))